## Training data-efficient image transformers & distillation through attention

Vision Transformers (ViT) are data hungry, to work well they were trained on a private dataset with 300 Images (JFT-300M) then finetuned for a different doenstream task.

Data Efficient Image Transformers solves this problem by using **Knowledge Distillation** from another architecture (teacher) to a Transformer based Architecture (student) while using only one dataset (Imagenet)

From the paper; page 2:

In summary, our work makes the following contributions:
* We show that our neural networks that contains no convolutional layer
can achieve competitive results against the state of the art on ImageNet
with no external data. They are learned on a single node with 4 GPUs in
three days. Our two new models DeiT-S and DeiT-Ti have fewer parameters
and can be seen as the counterpart of ResNet-50 and ResNet-18.
* We introduce a new distillation procedure based on a distillation token,
which plays the same role as the class token, except that it aims at reproducing
the label estimated by the teacher. Both tokens interact in the
transformer through attention. This transformer-specific strategy outperforms
vanilla distillation by a significant margin.
* Interestingly, with our distillation, image transformers learn more from a
convnet than from another transformer with comparable performance.
* Our models pre-learned on Imagenet are competitive when transferred to
different downstream tasks such as fine-grained classification, on several
popular public benchmarks: CIFAR-10, CIFAR-100, Oxford-102 flowers,
Stanford Cars and iNaturalist-18/19.

### Knowledge Distillation

Knowledge Distillation (KD), introduced by Hinton et al. [24], refers to the
training paradigm in which a student model leverages “soft” labels coming
from a strong teacher network. This is the output vector of the teacher’s softmax
function rather than just the maximum of scores, wich gives a “hard” label.
Such a training improves the performance of the student model